In [65]:
import numpy as np

In [79]:
r_cam = 0.03
r_hum = 1
omg_cam = 4 * np.pi
omg_hum = 0.25 * np.pi

In [96]:
#time range
t_range = np.arange(0, 0.1 + 0.05, 0.05)
t_len = len(t_range)
#camera coordinate
x_cam, y_cam, z_cam = 10, 4, 5
#human coordinate
x_hum = y_hum = z_hum = 10
#camera trajectory
px_cam = x_cam + r_cam * np.cos(omg_cam * t_range + np.pi)
py_cam = y_cam + np.zeros(t_len)
pz_cam = z_cam + r_cam * np.sin(omg_cam * t_range + np.pi)
#human trajectory
px_hum = x_hum + r_hum *np.sin(omg_hum * t_range)
py_hum = y_hum + r_hum * np.cos(omg_hum * t_range)
pz_hum = z_hum + np.zeros(t_len)
#d trajectory
px_d = px_hum - px_cam
py_d = py_hum - py_cam
pz_d = pz_hum - py_cam
scale = 5431.2
px_d = px_d / scale
py_d = py_d / scale
pz_d = pz_d / scale

In [97]:
#evaluation
print(t_len)
print(px_cam[0:10])
print(px_d[0:10])

4
[ 9.97        9.97572949  9.99072949 10.00927051]
[5.52364118e-06 1.16972908e-05 1.61528954e-05 1.99342480e-05]


In [98]:
#motion calculation
cam_motion = np.zeros((t_len-1,3))
diff_motion = np.zeros((t_len-1,3))
for i in range(t_len-1):
    m_cam_k = np.zeros((1,3))
    m_diff_k = np.zeros((1,3))
    m_cam_k[0][0] = (px_cam[i+1] - px_cam[i]) / 0.05
    m_cam_k[0][1] = (py_cam[i+1] - py_cam[i]) / 0.05
    m_cam_k[0][2] = (pz_cam[i+1] - pz_cam[i]) / 0.05
    m_diff_k[0][0] = (px_d[i+1] - px_d[i]) / 0.05
    m_diff_k[0][1] = (py_d[i+1] - py_d[i]) / 0.05
    m_diff_k[0][2] = (pz_d[i+1] - pz_d[i]) / 0.05
    cam_motion[i] = m_cam_k
    diff_motion[i] = m_diff_k
mc_mean = cam_motion.mean(axis = 0)
md_mean = diff_motion.mean(axis = 0)

In [99]:
#evaluation
print(cam_motion[0:10])
print(diff_motion[0:10])
print(mc_mean)
print(md_mean)

[[ 0.1145898   0.         -0.35267115]
 [ 0.3         0.         -0.21796276]
 [ 0.37082039  0.          0.        ]]
[[ 1.23472992e-04 -2.83901811e-06  0.00000000e+00]
 [ 8.91120930e-05 -8.51267678e-06  0.00000000e+00]
 [ 7.56270514e-05 -1.41732095e-05  0.00000000e+00]]
[ 0.2618034  0.        -0.1902113]
[ 9.60707120e-05 -8.50830147e-06  0.00000000e+00]


In [100]:
def cov_cal(motion1, motion2, motion1_mean, motion2_mean):
    matrix_sum = np.zeros((3,3))
    for i in range(t_len-1):
        matrix_sum += (motion1[i] - motion1_mean).T * (motion2[i] - motion2_mean)
    result = matrix_sum/(t_len-2)
    return result
    
#correlation calculation
cov_md_mc = cov_cal(diff_motion, cam_motion, md_mean, mc_mean)
cov_mc_mc = cov_cal(cam_motion, cam_motion, mc_mean, mc_mean)

In [101]:
#evaluation
print(cov_md_mc)
print(cov_mc_mc)
cov_mc_mc * cov_mc_mc

[[-3.26424507e-06  0.00000000e+00  0.00000000e+00]
 [-3.26424507e-06  0.00000000e+00  0.00000000e+00]
 [-3.26424507e-06  0.00000000e+00  0.00000000e+00]]
[[0.01750776 0.         0.03167184]
 [0.01750776 0.         0.03167184]
 [0.01750776 0.         0.03167184]]


array([[0.00030652, 0.        , 0.00100311],
       [0.00030652, 0.        , 0.00100311],
       [0.00030652, 0.        , 0.00100311]])

In [102]:
#optimal scale S
A = cov_md_mc * cov_mc_mc
B = cov_md_mc * cov_md_mc
S = -A.sum()/B.sum()

In [103]:
#solution
print(S)
#这个结果当时间积累越长，也就是观测数据越多的时候越接近于理论值，也就是物体的真实scale

5363.495600132572
